In [12]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['local']

subjects_collection = db['collectrium_stokstad_models_subjects']
makers = []
ids = []
n = 0

for artist in subjects_collection.find({'_type': 'Collectrium::Stokstad::Models::Artist', 'terms.0': {'$exists': True}}):
    
    if artist['approved']:
        for x in artist['terms']:
            makers.append(x['text'])
            ids.append(x['_id'])   

In [13]:
class TireNode:
    
    def __init__ (self):
        self.val = None
        self.mark = False
        self.pointers = {}
        
class Tire:
    
    def __init__ (self):
        self.root = TireNode()
        
    def insert(self, name):
        self.re_insert(name, self.root)
        return
    
    def re_insert(self, name, root):
        if name[0] not in root.pointers:
            newNode = TireNode()
            newNode.val = name[0]
            root.pointers[name[0]] = newNode
            self.re_insert(name, root)
        else:
            nextNode = root.pointers[name[0]]
            if(len(name[1:]) == 0):
                nextNode.mark = True
                return
            return self.re_insert(name[1:], nextNode)
        
    def search(self, name):
        if len(name) == 0:
            return False
        else:
            return self.re_search(name, self.root)
        
    def re_search(self, name, root):
        if name[0] not in root.pointers:
            return False
        else:
            nextNode = root.pointers[name[0]]
            if len(name[1:]) == 0:
                if nextNode.mark == True:
                    return True
                else:
                    return False
            else:
                return self.re_search(name[1:], nextNode)

In [14]:
def buildTree(names):
    
    tree = Tire()
    for x in names:
        tree.insert(x.split(' '))
    
    return tree

In [15]:
def searchTerms(sentence, tree):
    
    senten = sentence.replace("'", '').replace("\t", '').replace(',', '').lstrip().rstrip().lower()
    terms = senten.split(' ')
    oterms = sentence.lstrip().rstrip().split(' ')
    
    for x in range(len(terms)):  
        for y in range(x, len(terms)):
            tomatch = terms[x:y+1]
            if tree.search(tomatch):
                return ' '.join(oterms[x:y+1])
            else:
                return ''

In [16]:
import pandas as pd
import numpy as np
import collections
from collections import OrderedDict

newData = pd.read_csv("listings_1461025592.csv", warn_bad_lines=True, error_bad_lines=False)
oldData = pd.read_csv("sothebys_historical_data.csv", warn_bad_lines=True, error_bad_lines=False)

data = newData.append(oldData)
nullMaker = data[pd.isnull(data['Maker'])]

print len(makers)

clean = []
for x in makers:
    clean.append(x.replace("'", '').replace("\t", '').replace(',', '').lstrip().rstrip().lower())

# clean = sorted(list(set(clean)))

print len(clean)

Skipping line 65972: expected 18 fields, saw 19

Skipping line 225573: expected 18 fields, saw 27



834227
834227


In [17]:
tree = buildTree(clean)
makers = []

for index, row in nullMaker.iterrows():
    terms = row['Lot Title']
    try:
        maker = searchTerms(terms, tree)
        makers.append(maker)
    except:
        makers.append('')

In [18]:
nullMaker['Maker'] = makers
noMaker = nullMaker[nullMaker['Maker'] == '']
len(noMaker)/float(len(data))

/Users/gzhao/Library/Python/2.7/lib/python/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


0.7600087906007506

In [19]:
counter = collections.Counter(makers)
print counter.most_common(100)

[('', 681281), ('Three', 2298), ('THREE', 892), ('Group', 424), ('Statuette', 377), ('GROUP', 370), ('Multiple', 290), ('AQUAMARINE', 232), ('Commode', 204), ('ENAMEL', 189), ('Miscellaneous', 182), ('Unmounted', 162), ('Aquamarine', 155), ('Fauteuil', 129), ('Bureau', 109), ('Tazza', 109), ('Enamel', 106), ('OPAL', 103), ('ENAMEL,', 96), ('MISCELLANEOUS', 92), ('Miniature', 92), ('TURQUOISE', 82), ('UNMOUNTED', 81), ('Netsuke', 72), ('Turquoise', 68), ('CORAL', 66), ('CITRINE', 65), ('Opal', 61), ('Tourmaline', 59), ('TOURMALINE', 54), ('tazza', 50), ('PERIDOT', 50), ('GARNET', 46), ('Enamel,', 43), ('three', 43), ('Tanzanite', 42), ('Piccolo', 42), ('Statuette,', 41), ('Head', 40), ('Coral', 39), ('multiple', 38), ('ALEXANDRITE', 37), ('FIGURES', 36), ('Gong', 36), ('TOPAZ', 36), ('Dirk', 36), ('Boss', 35), ('Citrine', 35), ('Peridot', 34), ('Garnet', 32), ('Hardstone', 32), ('Military', 31), ('CONCH', 31), ('TURQUOISE,', 30), ('statuette', 30), ('MOONSTONE', 29), ('Armoire', 29), ('

In [4]:
x = [1, 2, 3]
print x[3:4]

[]
